# Information Value ENEM2019

## 1. Imports

In [4]:
import pandas as pd
import os 
import numpy as np 
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl

pd.set_option('display.max_columns', None)
sns.set_palette('Paired')
sns.set_style("whitegrid")
plt.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['font.size'] = 14

## 2. Reading Data

In [5]:
processed_directory = sorted(filter(lambda dir: '.parquet' in dir, os.listdir('../data/processed')), reverse = True)[0]
processed_directory = '../data/processed/' + processed_directory

data = pd.read_parquet(processed_directory).query('enem_year == 2019')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50873 entries, 54823 to 105695
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           50873 non-null  int64  
 1   enem_year                    50873 non-null  int64  
 2   age                          50873 non-null  object 
 3   sex                          50873 non-null  object 
 4   marital_status               50873 non-null  object 
 5   color_race                   50873 non-null  object 
 6   high_school_status           50873 non-null  object 
 7   year_completion_high_school  50873 non-null  int64  
 8   school_type                  50873 non-null  object 
 9   id_city_school               11461 non-null  float64
 10  city_school                  11461 non-null  object 
 11  state_school                 11461 non-null  object 
 12  id_city_test                 50873 non-null  int64  
 13  city_test  

In [6]:
cities = pd.read_csv('../data/external/cities.csv', sep = ';', usecols = ['ibge_code', 'lat_city', 'long_city', 'capital', 'state_lat', 'state_long', 'region'])
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ibge_code   5570 non-null   int64  
 1   lat_city    5570 non-null   float64
 2   long_city   5570 non-null   float64
 3   capital     5570 non-null   int64  
 4   state_lat   5570 non-null   float64
 5   state_long  5570 non-null   float64
 6   region      5570 non-null   object 
dtypes: float64(4), int64(2), object(1)
memory usage: 304.7+ KB


In [7]:
# Joining the candidates data and cities info
data = data.merge(cities, left_on = 'id_city_test', right_on = 'ibge_code')

In [8]:
# filter out candidates that failed in ENEM in one of the two days
# each presence day has value 1 then (presence_day_1 + presence_day_2) is equal 2
present_candidates = data.query('(presence_day_1 + presence_day_2) == 2')
present_candidates.head(5)

,id,enem_year,age,sex,marital_status,color_race,high_school_status,year_completion_high_school,school_type,id_city_school,city_school,state_school,id_city_test,city_test,state_test,presence_day_1,presence_day_2,nature_science_score,human_sciences_score,languages_codes_score,mathematics_score,ortography_score,understanding_score,structuring_score,argumentation_score,proposal_score,redaction_score,father_schooling,mother_schooling,father_profession_group,mother_profession_group,family_members,family_income,bedrooms_qty,cars_qty,television_qty,phones_qty,computer_qty,internet_access,ibge_code,lat_city,long_city,capital,state_lat,state_long,region
0,190001004687,2019,18 years-old,F,Single,Brown,Completed High School,1,Unknown,NaN,None,None,1500800,Ananindeua,PA,1.0,1.0,349.8,417.1,416.6,468.3,120.0,40.0,40.0,120.0,0.0,320.0,Completed Elementary School,Completed Elementary School,Group C,Group B,4.0,Less than 1 minimum wage,Two,No,One,No,No,No,1500800,-1.36391,-48.3743,0,-3.79,-52.48,Norte
1,190001007110,2019,18 years-old,F,Single,White,Ends in the current year,0,Public,1500800.0,Ananindeua,PA,1500800,Ananindeua,PA,1.0,1.0,524.5,544.7,545.4,458.1,160.0,120.0,140.0,140.0,140.0,700.0,Completed High School,Didn't complete Elementary School,Group F,Group F,6.0,Between 1-1.5 minimum wages,Two,No,One,Four or more,One,Yes,1500800,-1.36391,-48.3743,0,-3.79,-52.48,Norte
2,190001007346,2019,20 years-old,F,Single,White,Completed High School,3,Unknown,NaN,None,None,1500800,Ananindeua,PA,1.0,1.0,521.4,514.7,615.5,699.8,120.0,120.0,140.0,160.0,140.0,680.0,Completed Elementary School,Completed College,Group C,Group D,4.0,Between 8-9 minimum wages,Three,No,Three,Three,One,Yes,1500800,-1.36391,-48.3743,0,-3.79,-52.48,Norte
4,190001007985,2019,Less than 17 years-old,M,Single,Brown,Will complete High School the following year,0,Unknown,NaN,None,None,1500800,Ananindeua,PA,1.0,1.0,598.0,613.6,592.0,709.7,160.0,200.0,160.0,160.0,200.0,880.0,Completed High School,Completed High School,Group D,Group D,5.0,Between 3-4 minimum wages,Three,One,One,Three,Two,No,1500800,-1.36391,-48.3743,0,-3.79,-52.48,Norte
5,190001009480,2019,Less than 17 years-old,F,Single,White,Will complete High School the following year,0,Unknown,NaN,None,None,1500800,Ananindeua,PA,1.0,1.0,527.1,705.7,656.8,603.3,160.0,120.0,140.0,180.0,140.0,740.0,Completed High School,Completed High School,Group B,Group B,4.0,Less than 1 minimum wage,One,No,One,No,No,No,1500800,-1.36391,-48.3743,0,-3.79,-52.48,Norte


## 2.1 Building proxy variable (Target)

In [9]:
present_candidates['approved'] = np.where(present_candidates.mathematics_score >= present_candidates.mathematics_score.quantile(0.90), 1, 0)
present_candidates.iloc[:, -5:]

/tmp/ipykernel_18550/2426088829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  present_candidates['approved'] = np.where(present_candidates.mathematics_score >= present_candidates.mathematics_score.quantile(0.90), 1, 0)


,capital,state_lat,state_long,region,approved
0,0,-3.79,-52.48,Norte,0
1,0,-3.79,-52.48,Norte,0
2,0,-3.79,-52.48,Norte,1
4,0,-3.79,-52.48,Norte,1
5,0,-3.79,-52.48,Norte,0
...,...,...,...,...,...
50866,0,-3.47,-65.10,Norte,0
50869,0,-8.38,-37.86,Nordeste,0
50870,0,1.99,-61.33,Norte,0
50871,0,1.99,-61.33,Norte,0


## 3. Information Value

Weight of evidence (WOE) and Information value (IV) are simple, yet powerful techniques to perform variable transformation and selection. These concepts have huge connection with the logistic regression modeling technique.

<br>
<img width="14%" src="https://miro.medium.com/max/768/1*6Aw782wiyiFtzvK7EOY8CA.png">
<br><br>
<img width="23%" src="https://miro.medium.com/max/1200/1*9Gi0fGyTpxfwM2TpV4GZQQ.png">
<br><br>
<img width="23%" src="https://miro.medium.com/max/1156/1*5S_5aAHWe0_knDGZUK3W8w.png">
<br><br>

Observations: 
* Each category (bin) should have at least 5% of the observations.
* Each category (bin) should be non-zero for both non-events and events.
* The WOE should be distinct for each category. Similar groups should be aggregated.
* The WOE should be monotonic, i.e. either growing or decreasing with the groupings.
* Missing values are binned separately.

References:
* https://sundarstyles89.medium.com/weight-of-evidence-and-information-value-using-python-6f05072e83eb
* https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html

In [16]:
def information_value(var, df = present_candidates):
    ''' 
    Function to calculate the information value of a variable
    '''
    
    df_iv = df.groupby(var).size().to_frame(name = '# count')
    df_iv['count%'] = df_iv['# count'] / df_iv['# count'].sum()
    df_iv['# Approved'] = df.groupby(var).approved.sum()
    df_iv['# Non Approved'] = df_iv['# count'] - df_iv['# Approved']
    df_iv['Approved%'] = df_iv['# Approved'] / df_iv['# count'] 
    df_iv['Non Approved%'] = df_iv['# Non Approved'] / df_iv['# count'] 
    df_iv['Approved% - Non Approved%'] = df_iv['Approved%'] - df_iv['Non Approved%']
    df_iv['Approved% / Non Approved%'] = df_iv['Approved%'] / df_iv['Non Approved%']
    df_iv['WOE'] = np.log(df_iv['Approved% / Non Approved%'])
    df_iv['IV'] = df_iv['Approved% - Non Approved%'] * df_iv['WOE']

    return df_iv, df_iv.IV.sum()

## 3.1 Feature Selection

In [11]:
features_candidates = present_candidates[[
    'age', 
    'sex', 
    'marital_status', 
    'color_race', 
    'high_school_status', 
    'year_completion_high_school', 
    'school_type', 
    'state_test', 
    'father_schooling', 
    'mother_schooling', 
    'father_profession_group', 
    'mother_profession_group',
    'family_income',
    'region'
]]

features_candidates.head(5)

,age,sex,marital_status,color_race,high_school_status,year_completion_high_school,school_type,state_test,father_schooling,mother_schooling,father_profession_group,mother_profession_group,family_income,region
0,18 years-old,F,Single,Brown,Completed High School,1,Unknown,PA,Completed Elementary School,Completed Elementary School,Group C,Group B,Less than 1 minimum wage,Norte
1,18 years-old,F,Single,White,Ends in the current year,0,Public,PA,Completed High School,Didn't complete Elementary School,Group F,Group F,Between 1-1.5 minimum wages,Norte
2,20 years-old,F,Single,White,Completed High School,3,Unknown,PA,Completed Elementary School,Completed College,Group C,Group D,Between 8-9 minimum wages,Norte
4,Less than 17 years-old,M,Single,Brown,Will complete High School the following year,0,Unknown,PA,Completed High School,Completed High School,Group D,Group D,Between 3-4 minimum wages,Norte
5,Less than 17 years-old,F,Single,White,Will complete High School the following year,0,Unknown,PA,Completed High School,Completed High School,Group B,Group B,Less than 1 minimum wage,Norte


In [12]:
features_candidates.columns.to_series() \
    .apply(lambda x: information_value(x)[1]) \
        .to_frame(name = 'information_value') \
            .sort_values('information_value', ascending = False)

,information_value
state_test,58.962559
age,42.309531
year_completion_high_school,26.266117
family_income,19.988589
mother_schooling,16.079464
father_schooling,13.903074
marital_status,12.311869
color_race,11.876382
father_profession_group,11.616379
mother_profession_group,10.742793


### 3.7.1 State

In [15]:
state_test_iv_df, state_test_iv = information_value('state_test')
print('\nInformation Value (state_test_iv): ', state_test_iv, end = '\n\n')
state_test_iv_df


Information Value (state_test_iv):  58.96255914307033



,# count,count%,# Approved,# Non Approved,Approved%,Non Approved%,Approved% - Non Approved%,Approved% / Non Approved%,WOE,IV
state_test,,,,,,,,,,
AC,282,0.007637,6,276,0.021277,0.978723,-0.957447,0.021739,-3.828641,3.665720
AL,659,0.017847,37,622,0.056146,0.943854,-0.887709,0.059486,-2.822022,2.505133
AM,758,0.020528,23,735,0.030343,0.969657,-0.939314,0.031293,-3.464376,3.254137
AP,323,0.008747,8,315,0.024768,0.975232,-0.950464,0.025397,-3.673131,3.491180
BA,2849,0.077154,182,2667,0.063882,0.936118,-0.872236,0.068241,-2.684703,2.341694
CE,2075,0.056193,195,1880,0.093976,0.906024,-0.812048,0.103723,-2.266027,1.840124
DF,602,0.016303,104,498,0.172757,0.827243,-0.654485,0.208835,-1.566209,1.025060
ES,682,0.018469,81,601,0.118768,0.881232,-0.762463,0.134775,-2.004146,1.528088
GO,1245,0.033716,109,1136,0.087550,0.912450,-0.824900,0.095951,-2.343921,1.933499


### 3.7.1 Family Income

In [ ]:
# family_income label ordered

income_order = [
  'No income',
  'Less than 1 minimum wage',
  'Between 1-1.5 minimum wages',
  'Between 1.5-2 minimum wages',
  'Between 2-2.5 minimum wages',
  'Between 2.5-3 minimum wages',
  'Between 3-4 minimum wages',
  'Between 4-5 minimum wages',
  'Between 5-6 minimum wages',
  'Between 6-7 minimum wages',
  'Between 7-8 minimum wages',
  'Between 8-9 minimum wages',
  'Between 9-10 minimum wages',
  'Between 10-12 minimum wages',
  'Between 12-15 minimum wages',
  'Between 15-20 minimum wages',
  'More than 20 minimum wages'
]

In [ ]:
family_income_iv_df, family_income_iv = information_value('family_income')
print('\nInformation Value (family_income_iv): ', family_income_iv, end = '\n\n')
family_income_iv_df


Information Value (family_income_iv):  19.9885888394243



,# count,count%,# Approved,# Non Approved,Approved%,Non Approved%,Approved% - Non Approved%,Approved% / Non Approved%,WOE,IV
family_income,,,,,,,,,,
Between 1-1.5 minimum wages,9116,0.246872,345,8771,0.037846,0.962154,-0.924309,0.039334,-3.235662,2.990751
Between 1.5-2 minimum wages,3590,0.097221,260,3330,0.072423,0.927577,-0.855153,0.078078,-2.550046,2.180680
Between 10-12 minimum wages,478,0.012945,179,299,0.374477,0.625523,-0.251046,0.598662,-0.513058,0.128801
Between 12-15 minimum wages,440,0.011916,195,245,0.443182,0.556818,-0.113636,0.795918,-0.228259,0.025938
Between 15-20 minimum wages,380,0.010291,185,195,0.486842,0.513158,-0.026316,0.948718,-0.052644,0.001385
Between 2-2.5 minimum wages,3558,0.096355,327,3231,0.091906,0.908094,-0.816189,0.101207,-2.290587,1.869551
Between 2.5-3 minimum wages,1683,0.045578,238,1445,0.141414,0.858586,-0.717172,0.164706,-1.803594,1.293487
Between 3-4 minimum wages,2381,0.064480,361,2020,0.151617,0.848383,-0.696766,0.178713,-1.721975,1.199814
Between 4-5 minimum wages,1586,0.042951,310,1276,0.195460,0.804540,-0.609079,0.242947,-1.414913,0.861795


### 3.7.2 Age

In [ ]:
age_iv_df, age_iv = information_value('age')
print('\nInformation Value (age): ', age_iv, end = '\n\n')
age_iv_df


Information Value (age):  42.30953146333097



,# count,count%,# Approved,# Non Approved,Approved%,Non Approved%,Approved% - Non Approved%,Approved% / Non Approved%,WOE,IV
age,,,,,,,,,,
17 years-old,7378,0.199805,889,6489,0.120493,0.879507,-0.759013,0.137001,-1.987766,1.508741
18 years-old,7179,0.194416,773,6406,0.107675,0.892325,-0.784650,0.120668,-2.114711,1.659308
19 years-old,4275,0.115772,425,3850,0.099415,0.900585,-0.801170,0.110390,-2.203739,1.765569
20 years-old,2864,0.077561,288,2576,0.100559,0.899441,-0.798883,0.111801,-2.191033,1.750378
21 years-old,1902,0.051508,181,1721,0.095163,0.904837,-0.809674,0.105171,-2.252164,1.823519
22 years-old,1508,0.040838,128,1380,0.084881,0.915119,-0.830239,0.092754,-2.377809,1.974149
23 years-old,1110,0.030060,67,1043,0.060360,0.939640,-0.879279,0.064238,-2.745164,2.413766
24 years-old,881,0.023859,74,807,0.083995,0.916005,-0.832009,0.091698,-2.389259,1.987885
25 years-old,746,0.020203,60,686,0.080429,0.919571,-0.839142,0.087464,-2.436533,2.044597


In [ ]:
color_race_iv_df, color_race_iv = information_value('color_race')
print('\nInformation Value (color_race): ', color_race_iv, end = '\n\n')
color_race_iv_df


Information Value (color_race):  11.87638244709725



,# count,count%,# Approved,# Non Approved,Approved%,Non Approved%,Approved% - Non Approved%,Approved% / Non Approved%,WOE,IV
color_race,,,,,,,,,,
Black,4573,0.123842,170,4403,0.037175,0.962825,-0.925651,0.038610,-3.254243,3.012292
Brown,16712,0.452581,1055,15657,0.063128,0.936872,-0.873743,0.067382,-2.697377,2.356816
Indigenous,216,0.005850,12,204,0.055556,0.944444,-0.888889,0.058824,-2.833213,2.518412
Unknown,781,0.021150,107,674,0.137004,0.862996,-0.725992,0.158754,-1.840401,1.336117
White,13753,0.372448,2244,11509,0.163164,0.836836,-0.673671,0.194978,-1.634869,1.101364
Yellow,891,0.024129,104,787,0.116723,0.883277,-0.766554,0.132147,-2.023837,1.551381


### 3.7.4 Sex

In [ ]:
sex_iv_df, sex_iv = information_value('sex')
print('\nInformation Value (sex): ', sex_iv, end = '\n\n')
sex_iv_df


Information Value (sex):  3.495821001581836



,# count,count%,# Approved,# Non Approved,Approved%,Non Approved%,Approved% - Non Approved%,Approved% / Non Approved%,WOE,IV
sex,,,,,,,,,,
F,21996,0.595678,1523,20473,0.069240,0.930760,-0.861520,0.074391,-2.598425,2.238596
M,14930,0.404322,2169,12761,0.145278,0.854722,-0.709444,0.169971,-1.772127,1.257225


### 3.7.5 Features Candidates

In [ ]:
features_candidates = data[[
    'age', 
    'sex', 
    'marital_status', 
    'color_race', 
    'high_school_status', 
    'year_completion_high_school', 
    'school_type', 
    'state_test', 
    'father_schooling', 
    'mother_schooling', 
    'father_profession_group', 
    'mother_profession_group',
    'family_income',
    'region'
]]

features_candidates

,age,sex,marital_status,color_race,high_school_status,year_completion_high_school,school_type,state_test,father_schooling,mother_schooling,father_profession_group,mother_profession_group,family_income,region
0,18 years-old,F,Single,Brown,Completed High School,1,Unknown,PA,Completed Elementary School,Completed Elementary School,Group C,Group B,Less than 1 minimum wage,Norte
1,18 years-old,F,Single,White,Ends in the current year,0,Public,PA,Completed High School,Didn't complete Elementary School,Group F,Group F,Between 1-1.5 minimum wages,Norte
2,20 years-old,F,Single,White,Completed High School,3,Unknown,PA,Completed Elementary School,Completed College,Group C,Group D,Between 8-9 minimum wages,Norte
3,Between 26 and 30 years-old,M,Single,Brown,Completed High School,1,Unknown,PA,Completed High School,Completed Elementary School,Group C,Group B,Less than 1 minimum wage,Norte
4,Less than 17 years-old,M,Single,Brown,Will complete High School the following year,0,Unknown,PA,Completed High School,Completed High School,Group D,Group D,Between 3-4 minimum wages,Norte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50868,18 years-old,M,Single,Black,Ends in the current year,0,Public,RO,Never studied,Completed Elementary School,Group A,Group B,Less than 1 minimum wage,Norte
50869,Less than 17 years-old,F,Single,Black,Will complete High School the following year,0,Unknown,PE,Completed Graduate,Completed Elementary School,Group D,Group C,Between 3-4 minimum wages,Nordeste
50870,17 years-old,F,Single,Brown,Ends in the current year,0,Public,RR,Didn't complete Elementary School,Completed College,Group C,Group D,Between 1.5-2 minimum wages,Norte
50871,22 years-old,M,Single,Brown,Completed High School,3,Unknown,RR,Unknown,Completed High School,Group A,Group B,Between 1.5-2 minimum wages,Norte


In [ ]:
features_candidates.columns.to_series() \
    .apply(lambda x: information_value(x)[1]) \
        .to_frame(name = 'information_value') \
            .sort_values('information_value', ascending = False)

,information_value
state_test,58.962559
age,42.309531
year_completion_high_school,26.266117
family_income,19.988589
mother_schooling,16.079464
father_schooling,13.903074
marital_status,12.311869
color_race,11.876382
father_profession_group,11.616379
mother_profession_group,10.742793
